# Model Optimisation

This notebook performs hyperparameter optimisation for the six classical ML models across the nine feature-sets:
- RandomizedSearchCV
- GridSearchCV
- BayesSearchCV

In [1]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from scipy.stats import randint, uniform

FEATURES_BASE = Path("../data/processed/features1")
PROC_BASE = Path("../data/processed/ml1")
MODEL_BASE = Path("../models/ml1")
FIG_BASE = Path("../figures/ml1")

for p in [PROC_BASE, MODEL_BASE, FIG_BASE]:
    p.mkdir(parents=True, exist_ok=True)

METHODS = ["rfe","skb","fscs","etc","pc","mi","mir","mu","vt"]

OPT_MAP = {
    "rfe": "orfe",
    "skb": "oskb",
    "fscs": "ofscs",
    "etc": "oetc",
    "pc": "opc",
    "mi": "omi",
    "mir": "omir",
    "mu": "omu",
    "vt": "ovt"
}

RANDOM_STATE = 42
CV = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

try:
    from skopt import BayesSearchCV  # older style
except Exception:
    try:
        from skopt import BayesSearchCV
    except Exception as e:
        raise ImportError("skopt (scikit-optimize) not found. Install scikit-optimize to use Bayesian optimisation.") from e

print("Configuration OK. Methods:", METHODS)

Configuration OK. Methods: ['rfe', 'skb', 'fscs', 'etc', 'pc', 'mi', 'mir', 'mu', 'vt']


## Hyperparameter Spaces

In [2]:
PARAM_DIST = {
    "Logistic Regression": {
        "C": uniform(0.001, 10),
        "penalty": ["l2"],
        "solver": ["lbfgs", "saga"],
        "max_iter": [500]
    },
    "Gradient Boosting": {
        "n_estimators": randint(50, 300),
        "learning_rate": uniform(0.01, 0.5),
        "max_depth": randint(2, 8),
        "subsample": uniform(0.5, 0.5)
    },
    "KNN": {
        "n_neighbors": randint(1, 31),
        "weights": ["uniform", "distance"],
        "p": [1, 2]
    },
    "Random Forest": {
        "n_estimators": randint(50, 400),
        "max_depth": randint(3, 20),
        "max_features": ["sqrt", "log2", None]
    },
    "Decision Tree": {
        "max_depth": randint(1, 20),
        "min_samples_split": randint(2, 20),
        "min_samples_leaf": randint(1, 20)
    },
    "SVM": {
        "C": uniform(0.01, 100),
        "kernel": ["rbf", "poly"],
        "gamma": ["scale", "auto"]
    }
}

PARAM_GRID = {
    "Logistic Regression": {
        "C": [0.01, 0.1, 1, 10],
        "solver": ["lbfgs"],
        "penalty": ["l2"]
    },
    "Gradient Boosting": {
        "n_estimators": [100, 200],
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 5]
    },
    "KNN": {
        "n_neighbors": [3,5,7,9],
        "weights": ["uniform","distance"],
        "p": [1,2]
    },
    "Random Forest": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20]
    },
    "Decision Tree": {
        "max_depth": [None, 5, 10],
        "min_samples_split": [2, 5, 10]
    },
    "SVM": {
        "C": [0.1, 1, 10],
        "kernel": ["rbf"],
        "gamma": ["scale"]
    }
}

BAYES_SPACE = {
    "Logistic Regression": {
        "C": (1e-3, 100.0, "log-uniform"),
        "solver": ["lbfgs"],
        "penalty": ["l2"]
    },
    "Gradient Boosting": {
        "n_estimators": (50, 300),
        "learning_rate": (0.01, 0.5, "log-uniform"),
        "max_depth": (2, 8)
    },
    "KNN": {
        "n_neighbors": (1, 31),
        "weights": ["uniform", "distance"],
        "p": [1, 2]
    },
    "Random Forest": {
        "n_estimators": (50, 400),
        "max_depth": (3, 30)
    },
    "Decision Tree": {
        "max_depth": (1, 30),
        "min_samples_split": (2, 50)
    },
    "SVM": {
        "C": (1e-2, 100.0, "log-uniform"),
        "kernel": ["rbf"],
        "gamma": ["scale", "auto"]
    }
}

## Helper Functions

In [3]:
def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    rec = recall_score(y_true, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    return {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}

def save_confusion_matrix(y_true, y_pred, out_path, title):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel("Predicted"); ax.set_ylabel("True"); ax.set_title(title)
    fig.savefig(out_path, dpi=300, bbox_inches="tight")
    plt.close(fig)

def run_optimizer(name, model, param_space, X_train, y_train, optimizer_type="random", n_iter=30):
    if optimizer_type == "random":
        search = RandomizedSearchCV(model, param_distributions=param_space, n_iter=n_iter, cv=CV,
                                    scoring="accuracy", n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
    elif optimizer_type == "grid":
        search = GridSearchCV(model, param_grid=param_space, cv=CV, scoring="accuracy", n_jobs=-1, verbose=0)
    elif optimizer_type == "bayes":
        # BayesSearchCV from skopt
        search = BayesSearchCV(model, search_spaces=param_space, n_iter=n_iter, cv=CV, scoring="accuracy",
                               n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
    else:
        raise ValueError("Unknown optimizer_type")
    start = time.time()
    search.fit(X_train, y_train)
    end = time.time()
    return search, end - start

## Model Definitions

In [4]:
MODELS = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    "Gradient Boosting": GradientBoostingClassifier(random_state=RANDOM_STATE),
    "KNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(random_state=RANDOM_STATE),
    "Decision Tree": DecisionTreeClassifier(random_state=RANDOM_STATE),
    "SVM": SVC(probability=True, random_state=RANDOM_STATE)
}

## Main Optimisation Loop

In [5]:
n_iter_rand = 30
n_iter_bayes = 30

for method in tqdm(METHODS, desc="Methods"):
    feat_dir = FEATURES_BASE / method
    if not feat_dir.exists():
        print(f"⚠️ Missing feature folder for {method}; skipping.")
        continue

    train_df = pd.read_csv(feat_dir / "train.csv").dropna(subset=["DepressionEncoded"])
    test_df  = pd.read_csv(feat_dir / "test.csv").dropna(subset=["DepressionEncoded"])
    X_train = train_df.drop(columns=["DepressionEncoded"]).values
    y_train = train_df["DepressionEncoded"].astype(int).values
    X_test  = test_df.drop(columns=["DepressionEncoded"]).values
    y_test  = test_df["DepressionEncoded"].astype(int).values

    opt_folder = OPT_MAP.get(method, method)
    proc_out_base = PROC_BASE / opt_folder
    model_out_base = MODEL_BASE / opt_folder
    fig_out_base = FIG_BASE / opt_folder
    for p in [proc_out_base, model_out_base, fig_out_base]:
        p.mkdir(parents=True, exist_ok=True)

    rand_results = []
    grid_results = []
    bayes_results = []

    for model_name, model in tqdm(MODELS.items(), desc=f"Models ({method})", leave=False):
        print(f"\n=== {method.upper()} :: {model_name} ===")

        pdist = PARAM_DIST.get(model_name, {})
        pgrid = PARAM_GRID.get(model_name, {})
        bspace = BAYES_SPACE.get(model_name, {})

        try:
            print("-> RandomizedSearchCV (n_iter=", n_iter_rand, ")")
            search_rand, t_rand = run_optimizer(model_name, model, pdist, X_train, y_train, optimizer_type="random", n_iter=n_iter_rand)
            best_rand = search_rand.best_estimator_
            y_pred = best_rand.predict(X_test)
            metrics_rand = compute_metrics(y_test, y_pred)
            metrics_rand.update({"Method": method, "Model": model_name, "Optimizer": "Randomized", "TimeSec": t_rand})
            rand_results.append(metrics_rand)

            joblib.dump(best_rand, model_out_base / f"{model_name.lower().replace(' ','_')}_randomized.pkl")
            save_confusion_matrix(y_test, y_pred, fig_out_base / f"{model_name.lower().replace(' ','_')}_randomized_confusion.png",
                                  f"{model_name} Randomized ({method})")
            print(" Randomized done. Metrics:", metrics_rand)
        except Exception as e:
            print(" RandomizedSearchCV failed for", model_name, ":", e)

        try:
            print("-> GridSearchCV")
            if pgrid:
                search_grid, t_grid = run_optimizer(model_name, model, pgrid, X_train, y_train, optimizer_type="grid")
                best_grid = search_grid.best_estimator_
                y_pred = best_grid.predict(X_test)
                metrics_grid = compute_metrics(y_test, y_pred)
                metrics_grid.update({"Method": method, "Model": model_name, "Optimizer": "Grid", "TimeSec": t_grid})
                grid_results.append(metrics_grid)

                joblib.dump(best_grid, model_out_base / f"{model_name.lower().replace(' ','_')}_grid.pkl")
                save_confusion_matrix(y_test, y_pred, fig_out_base / f"{model_name.lower().replace(' ','_')}_grid_confusion.png",
                                      f"{model_name} Grid ({method})")
                print(" Grid done. Metrics:", metrics_grid)
            else:
                print("  No grid defined for", model_name, " — skipping GridSearch")
        except Exception as e:
            print(" GridSearchCV failed for", model_name, ":", e)

        try:
            print("-> BayesSearchCV (n_iter=", n_iter_bayes, ")")
            search_bayes, t_bayes = run_optimizer(model_name, model, bspace, X_train, y_train, optimizer_type="bayes", n_iter=n_iter_bayes)
            best_bayes = search_bayes.best_estimator_
            y_pred = best_bayes.predict(X_test)
            metrics_bayes = compute_metrics(y_test, y_pred)
            metrics_bayes.update({"Method": method, "Model": model_name, "Optimizer": "Bayes", "TimeSec": t_bayes})
            bayes_results.append(metrics_bayes)

            joblib.dump(best_bayes, model_out_base / f"{model_name.lower().replace(' ','_')}_bayes.pkl")
            save_confusion_matrix(y_test, y_pred, fig_out_base / f"{model_name.lower().replace(' ','_')}_bayes_confusion.png",
                                  f"{model_name} Bayes ({method})")
            print(" Bayes done. Metrics:", metrics_bayes)
        except Exception as e:
            print(" BayesSearchCV failed for", model_name, ":", e)

    if rand_results:
        pd.DataFrame(rand_results).to_csv(proc_out_base / "randomized_search_results.csv", index=False)
    if grid_results:
        pd.DataFrame(grid_results).to_csv(proc_out_base / "grid_search_results.csv", index=False)
    if bayes_results:
        pd.DataFrame(bayes_results).to_csv(proc_out_base / "bayes_search_results.csv", index=False)

    print(f"\nSaved results for method: {method} -> {proc_out_base}")

Methods:   0%|          | 0/9 [00:00<?, ?it/s]

Models (rfe):   0%|          | 0/6 [00:00<?, ?it/s]


=== RFE :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8197530864197531, 'Precision': 0.8203152249782567, 'Recall': 0.8197530864197531, 'F1': 0.8199045561326264, 'Method': 'rfe', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 7.380143165588379}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.8197530864197531, 'Precision': 0.8203152249782567, 'Recall': 0.8197530864197531, 'F1': 0.8199045561326264, 'Method': 'rfe', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.09836983680725098}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.8197530864197531, 'Precision': 0.8203152249782567, 'Recall': 0.8197530864197531, 'F1': 0.8199045561326264, 'Method': 'rfe', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 11.871528387069702}

=== RFE :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.7975308641975308, 'Prec

Models (skb):   0%|          | 0/6 [00:00<?, ?it/s]


=== SKB :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8123456790123457, 'Precision': 0.8139160148728836, 'Recall': 0.8123456790123457, 'F1': 0.8126308413592551, 'Method': 'skb', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 3.1620588302612305}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.817283950617284, 'Precision': 0.8188226267061313, 'Recall': 0.817283950617284, 'F1': 0.8175521920734646, 'Method': 'skb', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.11912322044372559}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.817283950617284, 'Precision': 0.8188226267061313, 'Recall': 0.817283950617284, 'F1': 0.8175521920734646, 'Method': 'skb', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 12.527231454849243}

=== SKB :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.7851851851851852, 'Precisi

...


=== MIR :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8469135802469135, 'Precision': 0.8496186157431163, 'Recall': 0.8469135802469135, 'F1': 0.8474364893080755, 'Method': 'mir', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 3.883526563644409}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.8419753086419753, 'Precision': 0.8442444497543733, 'Recall': 0.8419753086419753, 'F1': 0.842294202534971, 'Method': 'mir', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.13074994087219238}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.8419753086419753, 'Precision': 0.8442444497543733, 'Recall': 0.8419753086419753, 'F1': 0.842294202534971, 'Method': 'mir', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 17.47688317298889}

=== MIR :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.7950617283950617, 'Precisi

Models (mu):   0%|          | 0/6 [00:00<?, ?it/s]


=== MU :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8049382716049382, 'Precision': 0.8062732694483871, 'Recall': 0.8049382716049382, 'F1': 0.8047177580609789, 'Method': 'mu', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 3.7921485900878906}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.8049382716049382, 'Precision': 0.8061685900982564, 'Recall': 0.8049382716049382, 'F1': 0.804893649820939, 'Method': 'mu', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.14293503761291504}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.8049382716049382, 'Precision': 0.8062732694483871, 'Recall': 0.8049382716049382, 'F1': 0.8047177580609789, 'Method': 'mu', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 14.20425796508789}

=== MU :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.7925925925925926, 'Precision'

Models (vt):   0%|          | 0/6 [00:00<?, ?it/s]


=== VT :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8320987654320988, 'Precision': 0.8356545007530123, 'Recall': 0.8320987654320988, 'F1': 0.8324635326963289, 'Method': 'vt', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 2.3381338119506836}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.8271604938271605, 'Precision': 0.8311948509594571, 'Recall': 0.8271604938271605, 'F1': 0.827344083091447, 'Method': 'vt', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.1368253231048584}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.8271604938271605, 'Precision': 0.8311948509594571, 'Recall': 0.8271604938271605, 'F1': 0.827344083091447, 'Method': 'vt', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 17.91623854637146}

=== VT :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.7975308641975308, 'Precision': 

## Combine all optimizer results into a single report

In [6]:
combined_all = []
for method in METHODS:
    opt_folder = OPT_MAP.get(method, method)
    proc_out_base = PROC_BASE / opt_folder
    for fname in ["randomized_search_results.csv", "grid_search_results.csv", "bayes_search_results.csv"]:
        f = proc_out_base / fname
        if f.exists():
            df = pd.read_csv(f)
            df["OptFolder"] = opt_folder
            df["SourceFile"] = fname
            combined_all.append(df)

if combined_all:
    combined_df = pd.concat(combined_all, ignore_index=True)
    combined_df.to_csv(PROC_BASE / "all_optimised_results_summary.csv", index=False)
    display(combined_df.sort_values(["OptFolder", "Model", "Optimizer"], ascending=True))
    print("Combined optimisation summary saved to:", PROC_BASE / "all_optimised_results_summary.csv")
else:
    print("No optimisation results found to combine.")

,Accuracy,Precision,Recall,F1,Method,Model,Optimizer,TimeSec,OptFolder,SourceFile
70,0.708642,0.707324,0.708642,0.707446,etc,Decision Tree,Bayes,13.267796,oetc,bayes_search_results.csv
64,0.679012,0.707191,0.679012,0.685288,etc,Decision Tree,Grid,0.054684,oetc,grid_search_results.csv
58,0.708642,0.710961,0.708642,0.708705,etc,Decision Tree,Randomized,0.116686,oetc,randomized_search_results.csv
67,0.800000,0.801147,0.800000,0.798428,etc,Gradient Boosting,Bayes,126.297749,oetc,bayes_search_results.csv
61,0.782716,0.783921,0.782716,0.783217,etc,Gradient Boosting,Grid,16.004830,oetc,grid_search_results.csv
...,...,...,...,...,...,...,...,...,...,...
153,0.795062,0.799378,0.795062,0.793636,vt,Random Forest,Grid,2.260405,ovt,grid_search_results.csv
147,0.792593,0.800769,0.792593,0.793288,vt,Random Forest,Randomized,12.839989,ovt,randomized_search_results.csv
161,0.807407,0.811008,0.807407,0.806985,vt,SVM,Bayes,24.658028,ovt,bayes_search_results.csv
155,0.802469,0.805783,0.802469,0.802461,vt,SVM,Grid,1.020191,ovt,grid_search_results.csv


Combined optimisation summary saved to: ..\data\processed\ml1\all_optimised_results_summary.csv
